In [1]:
import string
import time
import gc
import json
import requests
import pandas as pd
import numpy as np
import Levenshtein as lev
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
def summary(data):
    return print('Shape:\n', data.shape,'\n\n',
                 'Columns:\n', data.columns,'\n\n',
                 'Data Types:\n', data.dtypes,'\n\n',
                 'Missing:\n', data.isnull().sum(),'\n\n',
                 'First ones:\n', data.head(10),'\n\n',
                 'Last ones:\n', data.tail(10),'\n\n',
                 'Stats:\n', data.describe([0.05,0.25,0.5,0.75,0.95])
                )

In [87]:
data = pd.read_csv(
    'C:\\Users\\nahue\\Desktop\\Problem 3 Input Data.txt',
    delimiter = '|',
    encoding = 'ISO-8859-1',
    quotechar = "'"
    )

In [4]:
summary(data)

Shape:
 (2794888, 2) 

 Columns:
 Index(['City', 'CountryCode'], dtype='object') 

 Data Types:
 City           object
CountryCode    object
dtype: object 

 Missing:
 City           25276
CountryCode      104
dtype: int64 

 First ones:
         City CountryCode
0   HANGZHOU          CN
1      JINAN          CN
2  GUANGZHOU          CN
3   HANGZHOU          CN
4   HANGZHOU          CN
5   BEIJINGI          CN
6   HANGZHOU          CN
7    BEIJING          CN
8   HANGZHOU          CN
9   HANGZHOU          CN 

 Last ones:
                City CountryCode
2794878   SINGAPORE          SG
2794879         NaN          SG
2794880   SINGAPORE          SG
2794881         NaN          SG
2794882   SINGAPORE          SG
2794883   SINGAPORE          SG
2794884   SINGAPORE          SG
2794885   SINGAPORE          SG
2794886   SINGAPORE          SG
2794887  HEIDELBERG          AU 

 Stats:
                 City CountryCode
count        2769612     2794784
unique         17113         205
top     I

In [5]:
world_cities = pd.read_csv(
    'C:\\Users\\nahue\\Desktop\\GEODATASOURCE-CITIES-FREE.txt',
    delimiter = '\t',
    encoding = 'ISO-8859-1',
    usecols = ['CC_ISO', 'FULL_NAME_ND']
    )

In [6]:
summary(world_cities)

Shape:
 (2828453, 2) 

 Columns:
 Index(['CC_ISO', 'FULL_NAME_ND'], dtype='object') 

 Data Types:
 CC_ISO          object
FULL_NAME_ND    object
dtype: object 

 Missing:
 CC_ISO          1918
FULL_NAME_ND       5
dtype: int64 

 First ones:
   CC_ISO       FULL_NAME_ND
0     AD              Aixas
1     AD         Aixirivall
2     AD           Aixovall
3     AD          Ansalonga
4     AD              Anyos
5     AD              Arans
6     AD            Arinsal
7     AD            Aubinya
8     AD         Bixessarri
9     AD  Bordes d'Envalira 

 Last ones:
         CC_ISO     FULL_NAME_ND
2828443     ZW       Westondale
2828444     ZW         Westwood
2828445     ZW      Whitecairns
2828446     ZW       Willowvale
2828447     ZW       Willsgrove
2828448     ZW  Wilmington Park
2828449     ZW     Windsor Park
2828450     ZW        Woodlands
2828451     ZW       Workington
2828452     ZW       Zimre Park 

 Stats:
          CC_ISO FULL_NAME_ND
count   2826535      2828448
unique      

In [7]:
def clean_words(words):
    words_ok = list(range(len(words)))
    to_remove = string.punctuation + '0123456789'
    table = str.maketrans('', '', to_remove)
    for i in range(len(words)):
        j = words.index[i]
        words_ok[i] = str(words[j]).translate(table)
        words_ok[i] = words_ok[i].lower()
    return words_ok

In [8]:
%%time
data_comp = data[data['City'].notnull()].drop_duplicates()
data_comp['City'] = clean_words(data_comp['City'])
wrong_cities = data_comp.drop_duplicates()
wrong_cities = wrong_cities.drop(wrong_cities[(wrong_cities['City'] == '')
                                             | (wrong_cities['City'] == ' ')
                                             | (wrong_cities['City'] == '  ')
                                             | (wrong_cities['City'] == '   ')
                                             ].index
                                )
cities_comp = world_cities[world_cities['FULL_NAME_ND'].notnull()
                          ].drop_duplicates()
cities_comp['FULL_NAME_ND'] = clean_words(cities_comp['FULL_NAME_ND'])
good_cities = cities_comp.drop_duplicates()

Wall time: 1min 12s


In [11]:
del data
del data_comp
del world_cities
del cities_comp
gc.collect()

60

In [10]:
%%time
right = 0
not_right = []
for i in wrong_cities['CountryCode'].unique():
    if i in good_cities['CC_ISO'].unique():
        right = right + 1
    else:
        not_right = not_right + [i]
print('Correct: ' + str(right))
print('Incorrect: ' + str(not_right))

Correct: 201
Incorrect: [nan, 'AN', 'VA']
Wall time: 34 s


In [12]:
wrong_cities[(wrong_cities['CountryCode'] == 'AN')
             | (wrong_cities['CountryCode'] == 'VA')
             ]

,City,CountryCode
1119461,marigot,AN
1479953,willemstad,AN
1511644,veghel,AN
1608025,willemstead,AN
2186664,fairfax,VA
2188184,sterling,VA
2422847,richmond,VA
2516432,saint martin,AN
2516763,st martin,AN
2520117,falls church,VA


In [13]:
for i in wrong_cities[(wrong_cities['CountryCode'] == 'AN')].index:
    wrong_cities['CountryCode'].loc[i] = 'NL'
for j in wrong_cities[(wrong_cities['CountryCode'] == 'VA')].index:
    wrong_cities['CountryCode'].loc[j] = 'US'
wrong_cities[(wrong_cities['CountryCode'] == 'AN')
             | (wrong_cities['CountryCode'] == 'VA')
             | (wrong_cities['CountryCode'].isnull())
             ]

,City,CountryCode
31126,jerusalem,NaN
51270,lviv,NaN
55049,windhoek,NaN
614741,ssa,NaN
1117912,hamburg,NaN
1571403,stpetersburg,NaN
1620651,moscow region,NaN
1623586,modin,NaN
1633485,moscow,NaN
1640747,st petersburg,NaN


In [14]:
for i in [31126, 51270, 1623586]:
    wrong_cities['CountryCode'].loc[i] = 'IL'
for j in [1571403, 1620651, 1633485, 1640747]:
    wrong_cities['CountryCode'].loc[j] = 'RU'
wrong_cities['CountryCode'].loc[614741] = 'BR'
wrong_cities['CountryCode'].loc[55049] = 'NA'
wrong_cities['CountryCode'].loc[1117912] = 'DE'
wrong_cities['CountryCode'].loc[1707579] = 'US'
wrong_cities['CountryCode'].loc[2513285] = 'GB'
wrong_cities[wrong_cities['CountryCode'].isnull()]

,City,CountryCode


In [15]:
wr_cities = wrong_cities.drop_duplicates()
del wrong_cities
gc.collect()

20

In [16]:
%%time
to_change = pd.DataFrame({'City': [],
                          'CountryCode': []
                          })
correct = pd.DataFrame({'City': [],
                        'CountryCode': []
                        })
for i in wr_cities['CountryCode'].unique():
    gd_subset = good_cities[good_cities['CC_ISO'] == i]
    wr_subset = wr_cities[wr_cities['CountryCode'] == i]
    for j in wr_subset.index:
        row = wr_subset.loc[j]
        if row['City'] in gd_subset['FULL_NAME_ND'].values:
            correct = correct.append(row)
        else:
            to_change = to_change.append(row)

Wall time: 1min 53s


In [17]:
to_change.describe()

,City,CountryCode
count,10925,10925
unique,10327,193
top,singapore,US
freq,12,1011


In [18]:
correct.describe()

,City,CountryCode
count,6886,6886
unique,6591,128
top,san juan,US
freq,5,1362


In [18]:
%%time
corrected = pd.DataFrame({'City': [],
                          'CountryCode': [],
                          'Suggested' : [],
                          'Score': []
                          })
for i in to_change['CountryCode'].unique():
    wr_subset = to_change[to_change['CountryCode'] == i]
    if i in correct['CountryCode'].unique():
        gd_subset = correct[correct['CountryCode'] == i]
        for j in wr_subset.index:
            row = wr_subset.loc[j]
            sugg = process.extractOne(row['City'],
                                      gd_subset['City'],
                                      scorer = lev.ratio
                                      )
            row['Suggested'] = sugg[0].title()
            row['Score'] = round(sugg[1], 2)
            corrected = corrected.append(row)
        del gd_subset
    else:
        wr_subset['Suggested'] = ['Missing C'] * len(wr_subset)
        wr_subset['Score'] = [0] * len(wr_subset)
        corrected = corrected.append(wr_subset)
    del wr_subset
    gc.collect()

C:\Users\nahue\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\nahue\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Wall time: 1min 12s


In [19]:
del to_change
gc.collect()

20

In [20]:
%%time
done = corrected[corrected['Score'] > 0.8][:]
change_again = corrected[corrected['Score'] <= 0.8][:]
done['New Suggestion'] = done['Suggested'][:]
done['New Score'] = done['Score'][:]
for i in change_again['CountryCode'].unique():
    wr_subset = change_again[change_again['CountryCode'] == i]
    if i in good_cities['CC_ISO'].unique():
        gd_subset = good_cities[good_cities['CC_ISO'] == i]
        for j in wr_subset.index:
            row = wr_subset.loc[j]
            sugg = process.extractOne(row['City'],
                                      gd_subset['FULL_NAME_ND'],
                                      scorer = lev.ratio
                                      )
            row['New Suggestion'] = sugg[0].title()
            row['New Score'] = round(sugg[1], 2)
            done = done.append(row)
        del gd_subset
    else:
        wr_subset['New Suggestion'] = ['Missing C'] * len(wr_subset)
        wr_subset['New Score'] = [0] * len(wr_subset)
        done = done.append(wr_subset)
    del wr_subset
    gc.collect()

C:\Users\nahue\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\nahue\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\nahue\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_

Wall time: 19min 16s


In [21]:
done[(done['Score']<=0.8) & (done['New Score'] > 0.75)].sample(30)

,City,CountryCode,Suggested,Score,New Suggestion,New Score
2167192,varranqueras chao,AR,Parana,0.43,Barranqueras,0.76
234562,penicuik,US,Phoenicia,0.71,Penick,0.86
899526,pozuelo de alarco,ES,El Palmar,0.54,Pozuelo De Alarcon,0.97
758064,hospitalet llobre,ES,Mollet Del Valles,0.47,Lhospitalet De Llobregat,0.83
1378711,virton,FR,Evron,0.73,Aviron,0.83
306321,buenos aaires,AR,Posadas,0.50,Buenos Aires Chico,0.77
731209,sudbury,CA,Surrey,0.62,Sunbury,0.86
1399896,antwerp,DE,Munster,0.57,Ganter,0.77
732078,borgaro,IT,Bergamo,0.71,Borgomaro,0.88
450932,garza garcia,MX,Guadalajara,0.52,Maria Garcia,0.83


In [22]:
del corrected
gc.collect()

40

In [23]:
new_correct = correct.append(done[(done['Score'] <= 0.8)
                                  & (done['New Score'] > 0.75)
                                  ][['CountryCode', 'New Suggestion']
                                   ].rename(columns = 
                                            {'New Suggestion': 'City'}
                                           )
                            ).drop_duplicates()
new_correct

C:\Users\nahue\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,City,CountryCode
0,hangzhou,CN
1,jinan,CN
2,guangzhou,CN
7,beijing,CN
14,harbin,CN
...,...,...
1466825,Estate Thomas,VI
1496342,Sanaya,YE
1501808,Brighton,BB
1528357,Basrah,IQ


In [25]:
%%time
country_done = done[done['New Score'] > 0.75][:]
change_country = done[done['New Score'] <= 0.75][:]
country_done['Code Suggestion'] = country_done['CountryCode'][:]
country_done['Third Suggestion'] = country_done['Suggested'][:]
country_done['Country Score'] = country_done['Score'][:]
for j in change_country.index:
    row = change_country.loc[j]
    sugg = process.extractOne(row['City'],
                              new_correct['City'],
                              scorer = lev.ratio
                             )
    # If there're two countries with the same name,
    # it will choose the first one
    row['Code Suggestion'] = new_correct[new_correct['City'] == sugg[0]
                                    ]['CountryCode'
                                     ].iloc[0]
    row['Third Suggestion'] = sugg[0].title()
    row['Country Score'] = round(sugg[1], 2)
    country_done = country_done.append(row)
del change_country
gc.collect()

C:\Users\nahue\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\nahue\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\nahue\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Wall time: 2min 40s


In [94]:
def match(db, metric, n, code, new):
    out = pd.DataFrame()
    if 'CountryCode' == code: 
        columns = ['City', code, new]
    else:
        columns = ['City', 'CountryCode', code, new]
    dummy = db[db[metric] > n][columns]
    out['Input_City'] = dummy['City'][:]
    out['Input_CountryCode'] = dummy['CountryCode']
    out['Output_City'] = dummy[new][:]
    out['Output_CountryCode'] = dummy[code]
    out['Output_BlankCity'] = ['No'] * len(dummy)
    return out
metric = ['Score', 'New Score', 'Country Score',
          'Score', 'New Score', 'Country Score' 
          ]
n = [0.8, 0.7, 0.9,
     0.5, 0.6, 0.85
    ]
code = ['CountryCode', 'CountryCode', 'Code Suggestion',
        'CountryCode', 'CountryCode', 'Code Suggestion'
       ]
new = ['Suggested', 'New Suggestion', 'Third Suggestion',
       'Suggested', 'New Suggestion', 'Third Suggestion'
      ]

In [96]:
%%time
output = pd.DataFrame()
db = country_done[:]
for i in range(len(n)):
    out = match(db, metric[i], n[i], code[i], new[i])
    output = output.append(out)
    db = db.drop(out.index)
    del out
    gc.collect()
out = pd.DataFrame()
dummy = db[db['Suggested'] == 'Missing C'
           ][['City', 'CountryCode',
              'Code Suggestion', 'Third Suggestion'
             ]
          ]
out['Input_City'] = dummy['City'][:]
out['Input_CountryCode'] = list(dummy['CountryCode'].values)
out['Output_City'] = dummy['Third Suggestion'][:]
out['Output_CountryCode'] = list(dummy['Code Suggestion'].values)
out['Output_BlankCity'] = ['No'] * len(dummy)
output = output.append(out)
db = db.drop(out.index)
del out
del dummy
gc.collect()
out = pd.DataFrame()
out['Input_City'] = db['City'][:]
out['Input_CountryCode'] = db['CountryCode']
out['Output_City'] = db['New Suggestion'][:]
out['Output_CountryCode'] = db['CountryCode']
out['Output_BlankCity'] = ['No'] * len(db)
output = output.append(out)
db = db.drop(out.index)
del out
gc.collect()

Wall time: 310 ms


In [100]:
del country_done
del new_correct
gc.collect()

80

In [102]:
output

,Input_City,Input_CountryCode,Output_City,Output_CountryCode,Output_BlankCity
5,beijingi,CN,Beijing,CN,No
22,shen yang,CN,Shenyang,CN,No
331,nangjing,CN,Nanjing,CN,No
2628,guanghzou,CN,Guangzhou,CN,No
4650,guang zhou,CN,Guangzhou,CN,No
...,...,...,...,...,...
385648,guetemala city,GT,Temala,GT,No
1838326,guatemala cityu,GT,El Cintul,GT,No
1984725,guatemala city,GT,Chahal Guatemala,GT,No
2516690,cayenne cedex,GF,Antonou Conde,GF,No


In [28]:
no = ['No'] * len(correct['City'])
table = pd.DataFrame({'Input_City' : correct['City'],
                      'Input_CountryCode' : correct['CountryCode'],
                      'Output_City' : correct['City'].str.title(),
                      'Output_CountryCode' : correct['CountryCode'],
                      'Output_BlankCity' : no,
                     })
table.head()

,Input_City,Input_CountryCode,Output_City,Output_CountryCode,Output_BlankCity
0,hangzhou,CN,Hangzhou,CN,No
1,jinan,CN,Jinan,CN,No
2,guangzhou,CN,Guangzhou,CN,No
7,beijing,CN,Beijing,CN,No
14,harbin,CN,Harbin,CN,No


In [29]:
output = output.append(table)

In [40]:
output = output.sort_index()

In [110]:
# Shanghi goes to Shanghai or Shanghu
# shangahai --> Gahai or Shanghai
# London --> Bordon or London City or South London, etc
# Paris --> Le petit paris, paris 01 something, etc
# Buenos Aires is missing in Argentina. It's province, not city
# Singapore, Glaswog, Montreal, Moscow, Amsterdam, Buenos Aires are US cities

In [43]:
to_geocode = output[['Output_City', 'Output_CountryCode']][:]
to_geocode = to_geocode.drop_duplicates()
to_geocode

,Output_City,Output_CountryCode
0,Hangzhou,CN
1,Jinan,CN
2,Guangzhou,CN
5,Beijing,CN
14,Harbin,CN
...,...,...
2789970,Toongabbie,AU
2790488,Karaikudi,IN
2790661,Emu Heights,AU
2792403,Brea,US


In [ ]:
# The code below was executed in two different
# days because there's a limitation with the API

In [187]:
%%time
lat = []
long = []
url = "https://us1.locationiq.com/v1/search.php"
parameters = {'key': '#######',
              'city': '',
              'country' : '',
              'limit': 1,
              'format': 'json'
              }
for i in to_geocode.index:
    city = to_geocode['Output_City'].loc[i]
    code = to_geocode['Output_CountryCode'].loc[i]
    parameters['city'] = city
    parameters['country'] = code
    response = requests.get(url, parameters)
    location = json.loads(response.text)
    if 'error' in location:
        lat = lat + [np.nan]
        long = long + [np.nan]
    else:
        lat = lat + [location[0]['lat']]
        long = long + [location[0]['lon']]
    del city
    del code
    gc.collect()
    time.sleep(0.6)
to_geocode['Latitude'] = lat
to_geocode['Longitude'] = long

Wall time: 1h 38min 34s


In [107]:
missing = to_geocode[
    to_geocode['Latitude'].isnull()
    ].drop(
        columns = ['Latitude', 'Longitude']
        )
geocoded = to_geocode.drop(missing.index)
missing.head()

,Output_City,Output_CountryCode
144143,Kipparing,AU
1372956,Randwick,AU
1380926,Caulfield South,AU
1381205,Taringa,AU
1382596,Kurralta Park,AU


In [130]:
%%time
lat = []
long = []
params ={'key': '#############',
         'address' : ''
        }
google_url = 'https://maps.googleapis.com/maps/api/geocode/json'
for i in missing.index:
    city = missing['Output_City'].loc[i]
    code = missing['Output_CountryCode'].loc[i]
    address = city + ', ' + code
    params['address'] = address
    response = requests.get(google_url, params)
    location = json.loads(response.text)
    if 'results' in location:
        if location['status'] == 'OK':
            res = location['results'][0]['geometry']['location']
            lat = lat + [res['lat']]
            long = long + [res['lng']]
        else:
            lat = lat + [np.nan]
            long = long + [np.nan]
    else:
        lat = lat + [np.nan]
        long = long + [np.nan]
    del city
    del code
    gc.collect()
missing['Latitude'] = lat
missing['Longitude'] = long

Wall time: 2min 6s


In [132]:
missing[missing['Latitude'].isnull()]

,Output_City,Output_CountryCode,Latitude,Longitude
1504692,Pusanjingu,KR,NaN,NaN
1523810,Monea,GB,NaN,NaN
38559,Moebis,DE,NaN,NaN
1441843,Chinnatambinadanur,IN,NaN,NaN
1402461,Merlin,BE,NaN,NaN
2536122,Sainteloi,BE,NaN,NaN
2130106,Kamapapila,ZA,NaN,NaN
856401,Grenadesurladour,FR,NaN,NaN
1579220,Pegazanes,ES,NaN,NaN
2147581,Gacia,ES,NaN,NaN


In [ ]:
# Those cities aren't in google!

In [134]:
geocoded = geocoded.append(missing)

In [137]:
del missing
gc.collect()

140

In [ ]:
merged = output.merge(geocoded)

In [138]:
merged.head()

,Output_City,Output_CountryCode,Input_City,Input_CountryCode,Output_BlankCity,Latitude,Longitude
0,Hangzhou,CN,hangzou,CN,No,30.248963,120.205234
1,Hangzhou,CN,ghangzhou,CN,No,30.248963,120.205234
2,Hangzhou,CN,hang zhou,CN,No,30.248963,120.205234
3,Hangzhou,CN,hanghzou,CN,No,30.248963,120.205234
4,Hangzhou,CN,hangzhouse,CN,No,30.248963,120.205234


In [139]:
merged.count()

Output_City           15353
Output_CountryCode    15353
Input_City            15353
Input_CountryCode     15353
Output_BlankCity      15353
Latitude              15334
Longitude             15334
dtype: int64

In [ ]:
# Re-load data from the first cells

In [88]:
%%time
data['Clean City'] = clean_words(data['City'])
data['Clean Code'] = data['CountryCode']
data.head()

Wall time: 1min 34s


In [89]:
data.head()

,City,CountryCode,Clean City,Clean Code
0,HANGZHOU,CN,hangzhou,CN
1,JINAN,CN,jinan,CN
2,GUANGZHOU,CN,guangzhou,CN
3,HANGZHOU,CN,hangzhou,CN
4,HANGZHOU,CN,hangzhou,CN


In [90]:
for i in data[(data['Clean Code'] == 'AN')].index:
    data['Clean Code'].loc[i] = 'NL'
for j in data[(data['Clean Code'] == 'VA')].index:
    data['Clean Code'].loc[j] = 'US'
for i in [31126, 51270, 1623586]:
    data['Clean Code'].loc[i] = 'IL'
for j in [1571403, 1620651, 1633485, 1640747]:
    data['Clean Code'].loc[j] = 'RU'
data['Clean Code'].loc[614741] = 'BR'
data['Clean Code'].loc[55049] = 'NA'
data['Clean Code'].loc[1117912] = 'DE'
data['Clean Code'].loc[1707579] = 'US'
data['Clean Code'].loc[2513285] = 'GB'

In [91]:
data[(data['Clean Code'] == 'AN')
     | (data['Clean Code'] == 'VA')
     ]

,City,CountryCode,Clean City,Clean Code


In [92]:
spaces = data[(data['Clean City'] == '')
              | (data['Clean City'] == ' ')
              | (data['Clean City'] == '  ')
              | (data['Clean City'] == '   ')
             ].index
for i in spaces:
    data['Clean City'].loc[i] = 'nan'

In [93]:
data[(data['Clean City'] == '')
     | (data['Clean City'] == ' ')
     | (data['Clean City'] == '  ')
     | (data['Clean City'] == '   ')
    ]

,City,CountryCode,Clean City,Clean Code


In [140]:
real_cities = merged[:]
for i in merged.index:
    row = merged.loc[i]
    if row['Output_City'].lower() in merged['Input_City']:
        real_cities = real_cities.append(row)
real_cities = real_cities.drop_duplicates()
table = pd.DataFrame({'Input_City' : real_cities['Output_City'].str.lower(),
                      'Input_CountryCode' : real_cities['Output_CountryCode'],
                      'Output_City' : real_cities['Output_City'].str.title(),
                      'Output_CountryCode' : real_cities['Output_CountryCode'],
                      'Output_BlankCity' : ['No'] * len(real_cities['Latitude']),
                      'Latitude' : real_cities['Latitude'],
                      'Longitude' : real_cities['Longitude'],
                     })
table.head()

,Input_City,Input_CountryCode,Output_City,Output_CountryCode,Output_BlankCity,Latitude,Longitude
0,hangzhou,CN,Hangzhou,CN,No,30.248963,120.205234
1,hangzhou,CN,Hangzhou,CN,No,30.248963,120.205234
2,hangzhou,CN,Hangzhou,CN,No,30.248963,120.205234
3,hangzhou,CN,Hangzhou,CN,No,30.248963,120.205234
4,hangzhou,CN,Hangzhou,CN,No,30.248963,120.205234


In [141]:
merged = merged.append(table)
merged.tail()

,Input_City,Input_CountryCode,Latitude,Longitude,Output_BlankCity,Output_City,Output_CountryCode
15348,toongabbie,AU,-33.786689,150.952650,No,Toongabbie,AU
15349,karaikudi,IN,10.068091,78.765701,No,Karaikudi,IN
15350,emu heights,AU,-33.737223,150.651105,No,Emu Heights,AU
15351,brea,US,33.917044,-117.888856,No,Brea,US
15352,diliman,PH,14.649583,121.059650,No,Diliman,PH


In [142]:
merged = merged.drop_duplicates()
merged.count()

Input_City            25856
Input_CountryCode     25856
Latitude              25820
Longitude             25820
Output_BlankCity      25856
Output_City           25856
Output_CountryCode    25856
dtype: int64

In [143]:
unique_nan = data[(data['Clean City'] == 'nan')
                 ]['Clean Code'
                  ].unique()
blank_city = ['Yes'] * len(unique_nan)
city = ['nan'] * len(unique_nan)
coordinates = np.nan * len(unique_nan)
table = pd.DataFrame({'Input_City' : city,
                      'Input_CountryCode' : unique_nan,
                      'Output_City' : city,
                      'Output_CountryCode' : unique_nan,
                      'Output_BlankCity' : blank_city,
                      'Latitude' : coordinates,
                      'Longitude' : coordinates,
                     })
table.head()

,Input_City,Input_CountryCode,Output_City,Output_CountryCode,Output_BlankCity,Latitude,Longitude
0,nan,SG,nan,SG,Yes,NaN,NaN
1,nan,US,nan,US,Yes,NaN,NaN
2,nan,DE,nan,DE,Yes,NaN,NaN
3,nan,CN,nan,CN,Yes,NaN,NaN
4,nan,HK,nan,HK,Yes,NaN,NaN


In [144]:
merged = merged.append(table)
merged.tail()

,Input_City,Input_CountryCode,Latitude,Longitude,Output_BlankCity,Output_City,Output_CountryCode
84,nan,SV,NaN,NaN,Yes,nan,SV
85,nan,KE,NaN,NaN,Yes,nan,KE
86,nan,BH,NaN,NaN,Yes,nan,BH
87,nan,GM,NaN,NaN,Yes,nan,GM
88,nan,CI,NaN,NaN,Yes,nan,CI


In [145]:
merged = merged.drop_duplicates()
merged.count()

Input_City            25945
Input_CountryCode     25944
Latitude              25820
Longitude             25820
Output_BlankCity      25945
Output_City           25945
Output_CountryCode    25944
dtype: int64

In [146]:
merged[merged['Input_CountryCode'].isnull()]

,Input_City,Input_CountryCode,Latitude,Longitude,Output_BlankCity,Output_City,Output_CountryCode
48,nan,NaN,NaN,NaN,Yes,nan,NaN


In [148]:
final = data.merge(merged,
                   how = 'left',
                  left_on = ['Clean City', 'Clean Code'],
                  right_on = ['Input_City', 'Input_CountryCode']
                  )
final.head(10)

,City,CountryCode,Clean City,Clean Code,Input_City,Input_CountryCode,Latitude,Longitude,Output_BlankCity,Output_City,Output_CountryCode
0,HANGZHOU,CN,hangzhou,CN,hangzhou,CN,30.248963,120.205234,No,Hangzhou,CN
1,JINAN,CN,jinan,CN,jinan,CN,30.443141,112.197889,No,Jinan,CN
2,GUANGZHOU,CN,guangzhou,CN,guangzhou,CN,23.130196,113.259294,No,Guangzhou,CN
3,HANGZHOU,CN,hangzhou,CN,hangzhou,CN,30.248963,120.205234,No,Hangzhou,CN
4,HANGZHOU,CN,hangzhou,CN,hangzhou,CN,30.248963,120.205234,No,Hangzhou,CN
5,BEIJINGI,CN,beijingi,CN,beijingi,CN,39.906217,116.391276,No,Beijing,CN
6,HANGZHOU,CN,hangzhou,CN,hangzhou,CN,30.248963,120.205234,No,Hangzhou,CN
7,BEIJING,CN,beijing,CN,beijing,CN,39.906217,116.391276,No,Beijing,CN
8,HANGZHOU,CN,hangzhou,CN,hangzhou,CN,30.248963,120.205234,No,Hangzhou,CN
9,HANGZHOU,CN,hangzhou,CN,hangzhou,CN,30.248963,120.205234,No,Hangzhou,CN


In [149]:
final.count()

City                  2769612
CountryCode           2794784
Clean City            2794888
Clean Code            2794796
Input_City            2794801
Input_CountryCode     2794723
Latitude              2769381
Longitude             2769381
Output_BlankCity      2794801
Output_City           2794801
Output_CountryCode    2794723
dtype: int64

In [150]:
data.count()

City           2769612
CountryCode    2794784
Clean City     2794888
Clean Code     2794796
dtype: int64

In [151]:
final = final.drop(columns = ['Clean City', 'Clean Code',
                              'Input_City', 'Input_CountryCode'
                             ]
                  )
final = final.rename(columns = {'City' : 'Input_City',
                                'CountryCode' : 'Input_CountryCode'
                               })
final.head()

,Input_City,Input_CountryCode,Latitude,Longitude,Output_BlankCity,Output_City,Output_CountryCode
0,HANGZHOU,CN,30.248963,120.205234,No,Hangzhou,CN
1,JINAN,CN,30.443141,112.197889,No,Jinan,CN
2,GUANGZHOU,CN,23.130196,113.259294,No,Guangzhou,CN
3,HANGZHOU,CN,30.248963,120.205234,No,Hangzhou,CN
4,HANGZHOU,CN,30.248963,120.205234,No,Hangzhou,CN


In [152]:
final = final[['Input_City',
               'Input_CountryCode',
               'Output_City',
               'Output_CountryCode',
               'Output_BlankCity',
               'Latitude',
               'Longitude',
              ]]
final = final.drop_duplicates()
final.shape

(18713, 7)

In [153]:
final.head()

,Input_City,Input_CountryCode,Output_City,Output_CountryCode,Output_BlankCity,Latitude,Longitude
0,HANGZHOU,CN,Hangzhou,CN,No,30.248963,120.205234
1,JINAN,CN,Jinan,CN,No,30.443141,112.197889
2,GUANGZHOU,CN,Guangzhou,CN,No,23.130196,113.259294
5,BEIJINGI,CN,Beijing,CN,No,39.906217,116.391276
7,BEIJING,CN,Beijing,CN,No,39.906217,116.391276


In [154]:
final.to_csv('C:\\Users\\nahue\\Desktop\\Output.txt',
              sep = '|',
              decimal = '.',
              index = False
              )